In [1]:
import keras
import os, shutil
from keras.applications import VGG16
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [2]:
conv_base = VGG16(weights='imagenet',
                  include_top=False, # 네트워크 최종 완전 연결 분류기 포함여부
                  input_shape=(150, 150, 3)) # 입력영상 크기. 설정하지 않으면 알아서 처리해줌

In [3]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [4]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = './datasets/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(directory, target_size=(150, 150), batch_size=batch_size, class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

In [5]:
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [6]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5), loss='binary_crossentropy', metrics=['acc'])

history = model.fit(train_features, train_labels, epochs=30, batch_size=20,
                   validation_data=(validation_features, validation_labels))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6030 - acc: 0.6655 - val_loss: 0.4549 - val_acc: 0.7910
Epoch 2/30
2000/2000 [==============================] - 3s 1ms/step - loss: 0.4342 - acc: 0.8010 - val_loss: 0.3917 - val_acc: 0.8180
Epoch 3/30
2000/2000 [==============================] - 3s 1ms/step - loss: 0.3663 - acc: 0.8340 - val_loss: 0.3251 - val_acc: 0.8720
Epoch 4/30
2000/2000 [==============================] - 3s 1ms/step - loss: 0.3153 - acc: 0.8695 - val_loss: 0.3026 - val_acc: 0.8850
Epoch 5/30
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2869 - acc: 0.8805 - val_loss: 0.2873 - val_acc: 0.8820
Epoch 6/30
2000/2000 [==============================] - 3s 1ms/step - lo

In [7]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('acc')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

# 검증 데이터는 증식되어서는 안됨
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
# fine tunning
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# trainable을 true로 설정해야 base network 훈련이 가능함
conv_base.trainable = True

# block5_conv1 다시 학습
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    # 타깃 디렉터리
    train_dir,
    # 모든 이미지의 크기를 150 * 150으로 변경
    target_size=(150, 150),
    batch_size=20,
    # binary_crossentropy 손실을 사용하므로 이진 레이블이 필요합니다
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [11]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=50)

model.save('cats_and_dogs_small_4.h5')

Epoch 1/50
100/100 [==============================] - 569s 6s/step - loss: 0.4639 - acc: 0.7935 - val_loss: 0.2931 - val_acc: 0.8910
Epoch 2/50
100/100 [==============================] - 567s 6s/step - loss: 0.2898 - acc: 0.8770 - val_loss: 0.2556 - val_acc: 0.8870
Epoch 3/50
100/100 [==============================] - 583s 6s/step - loss: 0.2490 - acc: 0.9005 - val_loss: 0.2352 - val_acc: 0.8940
Epoch 4/50
100/100 [==============================] - 567s 6s/step - loss: 0.2180 - acc: 0.9095 - val_loss: 0.2288 - val_acc: 0.9030
Epoch 5/50
100/100 [==============================] - 563s 6s/step - loss: 0.1911 - acc: 0.9245 - val_loss: 0.1828 - val_acc: 0.9240
Epoch 6/50
100/100 [==============================] - 567s 6s/step - loss: 0.1678 - acc: 0.9295 - val_loss: 0.1809 - val_acc: 0.9270
Epoch 7/50
100/100 [==============================] - 587s 6s/step - loss: 0.1491 - acc: 0.9385 - val_loss: 0.1765 - val_acc: 0.9250
Epoch 8/50
100/100 [==============================] - 565s 6s/step - 

In [12]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

Found 1000 images belonging to 2 classes.
test acc: 0.9359999942779541
